In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU, Activation, BatchNormalization, MaxPool2D
import data_preprocessing as dp

import models.training as train
import utilities

### Import data

In [2]:
reshaped_data = tf.data.experimental.load('dataset/preprocessed/reshaped_pca98_ds')
channel_data = tf.data.experimental.load('dataset/preprocessed/channel_pca98_ds')

### Split

In [3]:
reshaped_ds = dp.split_dataset(reshaped_data)
channel_ds = dp.split_dataset(channel_data)

In [4]:
channel_data

<_LoadDataset shapes: ((None, 44, 640, 1), (None, 4)), types: (tf.float64, tf.float32)>

### Dense Model

### CNN

In [5]:
class Conv(Model):
    def __init__(self):
        """
        init: constructor of model
        call: performs forward pass of model
        """
        super(Conv, self).__init__()

        self.layer_list = [
            #Conv2D(filters=32, kernel_size=(3,3), padding = 'same'),
            #BatchNormalization(),
            #LeakyReLU(),
            #MaxPool2D(),
            Conv2D(filters=16, kernel_size=(3,3), padding = 'same'),
            BatchNormalization(),
            LeakyReLU(),
            Dropout(0.3),
            MaxPool2D(),
            Conv2D(filters=8, kernel_size=(3,3), padding = 'same'),
            BatchNormalization(),
            LeakyReLU(),
            tf.keras.layers.GlobalAveragePooling2D(),
            Dense(4, activation='softmax')
        ]

    @tf.function
    def call(self, inputs) -> tf.Tensor:
        """ Computes a forward step with the given data
        Parameters
        ----------
        inputs : tf.Tensor
            the input for the model
        training : bool
            true if call has been made from train_step, which tells the dropout layer to be active
        Returns
        ----------
        x : tf.Tensor
            the output of the model
        """
        x = inputs
        for layer in self.layer_list:
            x = layer(x)
        return x

In [6]:
model = Conv()
model.build((None, 44, 640, 1))
model.summary()

Model: "conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  160       
_________________________________________________________________
batch_normalization (BatchNo multiple                  64        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  1160      
_________________________________________________________________
batch_normalization_1 (Batch multiple                  32     

In [ ]:
%matplotlib tk
# Initialize the loss-function
loss_func = tf.losses.CategoricalCrossentropy()
# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam()
# Initialize Train-Object
trainer = train.Trainer(model, channel_ds,
                        optimizer, loss_func, train.accuracy)
# Initialize Plotting-Object
grapher = utilities.TrainingGrapher(3,1, supxlabel='Epochs', axs_xlabels=[['Train Loss', 'Test Loss', 'Test Accuracy']])


for epoch in range(10):
    print("Epoch:", epoch, end='\r')
    trainer.train_epoch()
    grapher.update([trainer.losses['train'], trainer.losses['test'], trainer.accuracies['test']])